In [ ]:
#!/usr/bin/env python
# Import Splinter and BeautifulSoup
from splinter import Browser
import pandas as pd
from bs4 import BeautifulSoup as soup
import datetime as dt
from webdriver_manager.chrome import ChromeDriverManager

: 

In [ ]:
def scrape_all():
    
    #Initiate headless driver for deployment 
    
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=True)
    
    news_title, news_paragraph = mars_news(browser)
    
    # Run all scraping functions and store results in dictionary
    data = {
      "news_title": news_title,
      "news_paragraph": news_paragraph,
      "featured_image": featured_image(browser),
      "facts": mars_facts(),
      "last_modified": dt.datetime.now()
    
    }
    
    #Stop webdriver and return data
    browser.quit()
    return data

: 

: 

In [ ]:
# Set up Splinter
#executable_path = {'executable_path': ChromeDriverManager().install()}
#browser = Browser('chrome', **executable_path, headless=False)

def mars_news(browser):
    
    # Visit the mars nasa news site
    url = 'https://redplanetscience.com'
    browser.visit(url)

    # Optional delay for loading the page
    browser.is_element_present_by_css('div.list_text', wait_time=1)


    # Convert the browser html to a soup object
    html = browser.html
    news_soup = soup(html, 'html.parser')
    
# Add try/except for error handling

    try:

        slide_elem = news_soup.select_one('div.list_text')
        # Use the parent element to find the first `a` tag and save it as `news_title`
        news_title = slide_elem.find('div', class_='content_title').get_text()

        # Use the parent element to find the paragraph text
        news_p = slide_elem.find('div', class_='article_teaser_body').get_text()

    except AttributeError:
        
        return None, None
    
    return news_title, news_p

: 

In [ ]:

def featured_image(browser):

    # Visit URL
    url = 'https://spaceimages-mars.com'
    browser.visit(url)

    # Find and click the full image button
    full_image_elem = browser.find_by_tag('button')[1]
    full_image_elem.click()

    # Parse the resulting html with soup
    html = browser.html
    img_soup = soup(html, 'html.parser')

    # Add try/except for error handling
    
    try:
    
        # Find the relative image url
        img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
    
    except AttributeError:
    
        return None
    
    # Use the base URL to create an absolute URL
    img_url = f'https://spaceimages-mars.com/{img_url_rel}'
   
    return img_url


: 

In [ ]:
def mars_facts():
    
    # Add try/except for error handling 
    try:
    
        # Mars Facts
        df = pd.read_html('https://galaxyfacts-mars.com')[0]
    
    except BaseException:
        
        return None
    
    # Assign columns and set index of dataframe
    
    df.columns=['description', 'Mars', 'Earth']
    df.set_index('description', inplace=True)

    # Convert dataframe into HTML format, add bootstrap
    
    return df.to_html(classes="table table-striped")

: 

In [ ]:
if __name__ == "__main__":
    
    # If running as script, print scraped data
    print(scrape_all())


: 

: 